In [21]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Loading the Data
data = pd.read_excel('Online_Retail.xlsx')
data.head()


# Stripping extra spaces in the description
data['Description'] = data['Description'].str.strip()
  
# Dropping the rows without any invoice number
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
  
# Dropping all transactions which were done on credit
data = data[~data['InvoiceNo'].str.contains('C')]

# Transactions done in France
basket_France = (data[data['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Defining the hot encoding function to make the data suitable 
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

# Encoding the datasets
basket_France = basket_France.applymap(hot_encode)

# Building the model
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)


# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

C:\Users\wille\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


       InvoiceNo StockCode                      Description  Quantity  \
26        536370     22728        ALARM CLOCK BAKELIKE PINK        24   
27        536370     22727         ALARM CLOCK BAKELIKE RED        24   
28        536370     22726       ALARM CLOCK BAKELIKE GREEN        12   
29        536370     21724  PANDA AND BUNNIES STICKER SHEET        12   
30        536370     21883                  STARS GIFT TAPE        24   
...          ...       ...                              ...       ...   
541904    581587     22613      PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899      CHILDREN'S APRON DOLLY GIRL         6   
541906    581587     23254     CHILDRENS CUTLERY DOLLY GIRL         4   
541907    581587     23255  CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138     BAKING SET 9 PIECE RETROSPOT         3   

               InvoiceDate  UnitPrice  CustomerID Country  
26     2010-12-01 08:45:00       3.75     12583.0  France  
27 

C:\Users\wille\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
